<a href="https://colab.research.google.com/github/keb-cuevas/PSMDataScience/blob/main/Machine_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

if not os.path.exists('templates'):
  os.makedirs('templates')
if not os.path.exists('static'):
  os.makedirs('static')

###Install NGROK

In [ ]:
!pip install flask_ngrok

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://ngrok-agent.s3.amazonaws.com buster InRelease [17.1 kB]

###Code

In [ ]:
!ngrok authtoken "26QPrLYlH7HHU8o9yrppySGzULp_25ihv92Cir1MomqmBhFrW"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import pandas as pd
path = 'https://kebcuevas.com/wp-content/uploads/2022/03/seeds_dataset.txt'

df = pd.read_csv(path, sep= '\t', header= None,
                names=['area','perimeter','compactness','lengthOfKernel',
                       'widthOfKernel','asymmetryCoefficient',
                      'lengthOfKernelGroove','seedType'])

df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 209
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   area                  199 non-null    float64
 1   perimeter             199 non-null    float64
 2   compactness           199 non-null    float64
 3   lengthOfKernel        199 non-null    float64
 4   widthOfKernel         199 non-null    float64
 5   asymmetryCoefficient  199 non-null    float64
 6   lengthOfKernelGroove  199 non-null    float64
 7   seedType              199 non-null    float64
dtypes: float64(8)
memory usage: 14.0 KB


In [ ]:
X= df.drop('seedType', axis = 1)
y= df['seedType']

from sklearn.model_selection import KFold, cross_val_score, train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.30, random_state=42)

In [ ]:
#do not show warnings
import warnings
warnings.filterwarnings("ignore")

#import machine learning related libraries
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [ ]:
#create an array of models
models = []
models.append(("LR",LogisticRegression()))
models.append(("NB",GaussianNB()))
models.append(("RF",RandomForestClassifier()))
models.append(("SVC",SVC()))
models.append(("Dtree",DecisionTreeClassifier()))
models.append(("XGB",xgb.XGBClassifier()))
models.append(("KNN",KNeighborsClassifier()))

#measure the accuracy 
for name,model in models:
    kfold = KFold(n_splits=2)
    cv_result = cross_val_score(model,X_train,y_train, cv = kfold,scoring = "accuracy")
    print(name, cv_result)

LR [0.97142857 0.94202899]
NB [0.9        0.91304348]
RF [0.94285714 0.95652174]
SVC [0.94285714 0.89855072]
Dtree [0.84285714 0.89855072]
XGB [0.92857143 0.94202899]
KNN [0.94285714 0.88405797]


In [ ]:
xgb_model = xgb.XGBClassifier().fit(X_train, y_train)

import joblib

joblib.dump(xgb_model, "xgb.pkl")

['xgb.pkl']

In [ ]:
text = '''

<!DOCTYPE html>
<html>
    <head>
        <title>Seed Variety Classifier</title>
    </head>
    
    <body>
        <form name="form", method="POST", style="text-align: center;">
            <h1><strong>Wheat Seed Variety Classifier</strong></h1>
            <br>
            Area: <input type="number" step="0.01" name="area", placeholder="" required/>
            <br><br>
            Perimeter: <input type="number" step="0.01" name="perimeter", placeholder="" required/>
            <br><br>            
            Compactness: <input type="number" step="0.01" name="compactness", placeholder="" required/>
            <br><br>
            Kernel Length: <input type="number" step="0.01" name="lengthOfKernel", placeholder="" required/>
            <br><br>
            Kernel Width: <input type="number" step="0.01" name="widthOfKernel", placeholder="" required/>
            <br><br>
            Asymmetry Coefficient: <input type="number" step="0.01" name="asymmetryCoefficient", placeholder="" required/>
            <br><br>
            Grove Length: <input type="number" step="0.01" name="lengthOfKernelGroove", placeholder="" required/>
            <br><br>

            <button value="Submit">Identify</button>
        </form>
        <p style="text-align: center;">The seed variety is <strong>{{ output }}</strong></p>
        <p style="text-align: center;">This was created by <a href="https://twitter.com/kebcuevas">@kebcuevas</a></p>
    </body>
</html>
'''
file = open("templates/text.html","w")
file.write(text)
file.close()

In [ ]:
from numpy import float16
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
from flask import request

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/', methods=['GET', 'POST'])

def text():
  if request.method == 'POST':
    
    xgb = joblib.load("xgb.pkl")
    # Get values through input bars
    area = request.form.get("area")
    perimeter = request.form.get("perimeter")
    compactness = request.form.get("compactness")
    lengthOfKernel = request.form.get("lengthOfKernel")
    widthOfKernel = request.form.get("widthOfKernel")
    asymmetryCoefficient = request.form.get("asymmetryCoefficient")
    lengthOfKernelGroove = request.form.get("lengthOfKernelGroove")

    # Put inputs to dataframe
    X = pd.DataFrame([[area, perimeter, compactness, lengthOfKernel, 
                       widthOfKernel, asymmetryCoefficient, lengthOfKernelGroove]], 
                     columns = ["area", "perimeter", "compactness", "lengthOfKernel", 
                                "widthOfKernel", "asymmetryCoefficient", "lengthOfKernelGroove"])
    X = X.astype(float16)

    # Get prediction
    predict = xgb.predict(X)[0]

    if predict == 1.0:
      prediction = "Kama"
    
    elif predict == 2.0:
      prediction = "Rosa"

    elif predict == 3.0:
      prediction = "Canadian"

    else:
      prediction = "Error"

  else:
    prediction = 'Unknown'

  return render_template('text.html', output = prediction)

app.run(debug=False,host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://93e0-34-145-237-104.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
